In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from cvxpy import * 

In [2]:
def train_test_split(data, start_year):
    '''
    Split the data up into training and test periods
    
    Args: 
        data: a n stocks by t periods data-frame with a date-time index
        period: an integer representing which period that should be used
        
    Returns:
        tuple of training and testing data as pandas dataframes
        
    Usage: 
        training1, testing1 = train_test_split(stocks, 1)
    '''
    
    training = data[str(start_year):str(start_year + 4)]
    testing = data[str(start_year + 5)]
    return(training, testing)

In [3]:
def read_stocks():
    '''
    Get stock data for performing optimization. 
    Assumes data is in a t-by-n format with t months of observations
    along the rows and n stocks to choose from in columns.
    Assumes no missing values.
    
    Returns:
         Monthly returns data in a t-by-n formated dataframe with 
         a time-formatted index
    '''
    
    df = pd.read_csv('../../data/monthly_return.csv', names = ['s_' + str(x+1) for x in range(556)])
    df['date'] = pd.date_range('1/1/1986', periods=360, freq='M')
    df.set_index('date', inplace=True)
    
    return(df)

In [4]:
def read_factors3():
    '''
    Get fama and french factor data for performing optimization. 
    Assumes data is in a t-by-3 format with t months of observations
    along the rows and 3 factors to choose from in columns.
    Assumes no missing values.

    Returns:
         Monthly returns data in a t-by-n formated dataframe with 
         a time-formatted index
    '''
    
    df = pd.read_csv('../../data/F-F_Research_Data_3_Factors.csv'
                     , skiprows=3, parse_dates=True, nrows=1088)
    keep_dates = (df.loc[:,'Unnamed: 0'] >= 198601) & (df.loc[:,'Unnamed: 0'] <= 201512)
    df = df.loc[keep_dates, ['Mkt-RF', 'SMB', 'HML']] # Exclude risk free rate
    df['date'] = pd.date_range('1/1/1986', periods=360, freq='M')
    df.set_index('date', inplace=True)
    
    return(df)

In [5]:
def read_factors5():
    '''
    Get fama and french factor data for performing optimization. 
    Returns data is in a t-by-5 format with t months of observations
    along the rows and 5 factors to choose from in columns.
    Assumes no missing values.

    Returns:
         Monthly returns data in a t-by-5 formated dataframe with 
         a time-formatted index
    '''
    
    df = pd.read_csv('../../data/F-F_Research_Data_5_Factors.csv'
                     , skiprows=3, parse_dates=True, nrows=645)
    keep_dates = (df.loc[:,'Unnamed: 0'] >= 198601) & (df.loc[:,'Unnamed: 0'] <= 201512)
    df = df.loc[keep_dates, ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']] # Exclude risk free rate   
    df['date'] = pd.date_range('1/1/1986', periods=360, freq='M')
    df.set_index('date', inplace=True)
    
    return(df)

In [6]:
def fama_and_french(stocks, factors):
    '''
    Calculate factor loadings, factor covariance, and idiosyncratic risk 
    for a single training period
    
    Args:
        stocks: stocks for a single period as a t by n numpy array (this is training period)
        factors: factors for a single period as a t by k numpy array (this is training period)
    
    Returns:
        dictionary including factor loadings (n by k), factor covariance (k by k) and idiosyncratic risk (n by n) 
    '''
    
    # Number of stocks
    n = stocks.shape[1]
    
    # Number of factors
    k = factors.shape[1]
    
    # Factor loadings
    F = np.zeros(shape = (n, k + 1))

    # Idiosyncratic risk
    D = np.diag(np.zeros(shape = n))

    # Define the input for the regression
    X = factors
    X = sm.add_constant(X)

    # Loop through the stocks and calculate coefficients
    for i in range(n):

        # Select new stock each time
        y_i = stocks.iloc[:,i] 
        model_i = sm.OLS(y_i, X).fit() 

        # Including the alpha term and all k betas
        F[i,:] = model_i.params.values

        # Denominator is: (60 months) - (k factors) + (1 constant)
        D[i,i] = np.sum((y_i - model_i.predict(X))**2) / (60 - k + 1) 

    # The Factor Covariance Matrix
    Sigma_tilde = np.cov(F, rowvar=False)
    
    return(F, Sigma_tilde, D)

In [7]:
def get_equal_weights(n):
    '''
    Get a vector of equal weights
    
    Args:
        n: How many stocks are in your portfolio
        
    Returns:
        a n-by-1 matrix of 1/n weights
    '''
    
    x = np.asmatrix(np.ones(shape = (n, 1))) / n
    return(x)

In [8]:
def ret_cov_est(training):
    '''
    Estimate the returns and covariance for the training period.
    
    Args:
        training: a n stocks by t periods data-frame with a date-time index
    
    Returns:
        A tuple with a n-by-1 matrix of estimated returns 
        and a n-by-n matrix of estimated covariance. n is the number of stocks.
    '''
    r_hat = np.asmatrix(np.mean(training)).T
    Sigma = np.asmatrix(np.cov(training, rowvar=False))
    return(r_hat, Sigma)

In [9]:
def min_variance(r_hat, Sigma, tau, mu = None):
    '''
    For a given estimated return floor, get the weights that 
    minimize the variance with an l1 norm of the weights.
    
    Args:
        r_hat: estimated returns as a n-by-1 matrix
        Sigma: estimated covariance as a n-by-n matrix
        tau: tuning parameter. (Larger values promote more sparsity.)
        mu: the minimum return that the portfolio must beat
        
    Returns:
        optimal weights as a n-by-1 matrix
    '''
    
    # Define the variables
    n = r_hat.shape[0]
    x = Variable(n)
    ret = r_hat.T*x 
    risk = quad_form(x, Sigma)
    
    # Define the problem: Minimize variance for given returns threshold
    objective = Minimize(risk + tau*norm(x, 1))
    constraints = [sum_entries(x) == 1, x >= 0, ret >= mu]
    
    # Solve the problem
    prob = Problem(objective, constraints)
    prob.solve()
    
    # Get the values of interest
    minimal_risk = risk.value
    optimal_x = x.value
    
    # Handling rounding of x's
    optimal_x = np.around(optimal_x, decimals = 4)
    optimal_x =  np.asmatrix(optimal_x / sum(optimal_x))
    
    return(optimal_x)

In [10]:
def min_variance_factor(r_hat, F, Sigma_tilde, D, mu):
    
    # Define the variables
    n = F.shape[0]
    x = Variable(n)   # The weights
    f = F.T*x         # The factor loadings
    ret = r_hat.T*x 
    risk = quad_form(f, Sigma_tilde) + quad_form(x, D)
    
    # Solve the problem
    prob = Problem(Minimize(risk), [sum_entries(x) == 1, x >= 0, ret >= mu])
    prob.solve()
    
    # Get the values of interest
    minimal_risk = risk.value
    optimal_x = x.value
    
    # Handling rounding of x's
    optimal_x = np.around(optimal_x, decimals = 4)
    optimal_x =  np.asmatrix(optimal_x / sum(optimal_x))
    
    return(optimal_x)

In [11]:
def format_results(results, test, strategy, num_stocks):
    '''
    Format the results of the risk, returns into a data-frame 
    for plotting later
    
    Args: 
        results: numpy array with returns, risk results
        test: 1 or 0 depending on whether we are talking about in or 
                out of sample performance 
        strategy: string indicating which type of optimization approach was used
        num_stocks: the number of stocks selected for the portfolio
        
    Returns: 
        Dataframe with the following columns:
            return: the return for the portfolio
            risk: the risk for the portfolio
            strategy: see above for definition
            year: if training, the start year of the training period, 
                    if testing the evaluation year
            test: binary, 1 for out-of-sample, 0 for in-sample
            period: one of 6 values indicating which period of six years 
                        its in
            num_stocks: the number of stocks selected for the portfolio
    '''
    years = np.arange(1986, 2010 + 1)
    df = pd.DataFrame(results, columns = ['Return', 'Risk'])
    df['Sharpe'] = df['Return'] / df['Risk']
    df['strategy'] = strategy
    df['year'] = years + 6 if test == 1 else years
    df['test'] = test
    df['period'] = np.repeat(np.array([1, 2, 3, 4, 5]), 5)
    df['num_stocks'] = num_stocks
    return(df)

In [12]:
def risk_return(x, data):
    '''
    Calculate the risk, returns for a portfolio with a given set of weights
    
    Args:
        x: wieghts to be applied for the portfolio as a n-by-1 matrix
        data: an t-by-n pandas dataframe or numpy matrix.
        
    Returns:
        A tuple of two floats: one the risk and one the return
    '''
    
    r_hat = np.asmatrix(np.mean(data)).T
    Sigma = np.asmatrix(np.cov(data, rowvar=False))
    ret = (r_hat.T * x)[0,0]
    risk = (x.T * Sigma * x)[0,0]
    return(ret, risk)

In [13]:
# Set constants
START_YEAR = 1986
TAU = 1
NUM_PERIOD = 25

# Create empty matrices to store the results

# In sample results
results_train_equal = np.asmatrix(np.zeros(shape = (25, 2)))
results_train_min_var = np.asmatrix(np.zeros(shape = (25, 2)))
results_train_factor = np.asmatrix(np.zeros(shape = (25, 2)))

# Out of sample results
results_test_equal = np.asmatrix(np.zeros(shape = (25, 2)))
results_test_min_var = np.asmatrix(np.zeros(shape = (25, 2)))
results_test_factor = np.asmatrix(np.zeros(shape = (25, 2)))

# Monthly returns (out of sample)
monthly_returns_equal = np.asmatrix(np.zeros(shape = (12, 25)))
monthly_returns_min_var = np.asmatrix(np.zeros(shape = (12, 25)))
monthly_returns_factor = np.asmatrix(np.zeros(shape = (12, 25)))

monthly_returns = np.asmatrix(np.zeros(shape = (12*25, 4)))

num_min_var = np.asmatrix(np.zeros(shape = (25, 1)))
num_factor = np.asmatrix(np.zeros(shape = (25, 1)))

In [14]:
# Get data
factors3 = read_factors3()
factors5 = read_factors5()
stocks = read_stocks()

# Number of stocks
n = stocks.shape[1]

# Calculate the equal weights vector
x_equal = get_equal_weights(n)

for i in range(NUM_PERIOD):
    
    # Set the period starting point
    start_year = START_YEAR + i
    
    # Get train, test split
    training, testing = train_test_split(stocks, start_year)
    train_factor3, test_factor3 = train_test_split(factors3, start_year)
    train_factor5, test_factor5 = train_test_split(factors5, start_year)
    
    # 1. Calculate the risk, returns for naive approach (annual and monthly)
    results_train_equal[i,:] = risk_return(x_equal, training)
    results_test_equal[i,:] = risk_return(x_equal, testing)
    
    ret_equal = results_train_equal[i,0]
    
    # 2. Estimate the risk, returns from the training data
    r_hat, Sigma = ret_cov_est(training)
    
    # 3. Calulate the factor loadings, factor covariance matrix, and 
    # idiosyncratic risk from the training data
    F3, Sigma_tilde3, D3 = fama_and_french(training, train_factor3)
    F5, Sigma_tilde5, D5 = fama_and_french(training, train_factor5)
    
    # 4. Get the optimized weights
    # Note: I am using the naive portfolio for theshholds
    
    x_min_var = min_variance(r_hat = r_hat, Sigma = Sigma, tau = TAU, mu = ret_equal)
    print('Minimizing variance is complete')    
    x_factor3 = min_variance_factor(r_hat, F3, Sigma_tilde3, D3, mu = ret_equal)
    print('F-F 3 factors is complete')    
    x_factor5 = min_variance_factor(r_hat, F5, Sigma_tilde5, D5, mu = ret_equal)
    print('F-F 5 factors is complete')
    
    # 5. Calculate the risk, returns for the optimized approachs (GET RID OF THIS?)
    results_train_min_var[i,:] = risk_return(x_min_var, training)
    results_test_min_var[i,:] = risk_return(x_min_var, testing)
    results_train_factor[i,:] = risk_return(x_factor3, training)
    results_test_factor[i,:] = risk_return(x_factor3, testing)
    
    # 6. Calculate the monthly returns for each approach
    # monthly_returns_equal[:,i] = testing.values * x_equal
    # monthly_returns_min_var[:,i] = testing.values * x_min_var
    # monthly_returns_factor[:,i] = testing.values * x_factor    
    
    monthly_returns[(12*i):(12*i + 12),0] = testing.values * x_equal
    monthly_returns[(12*i):(12*i + 12),1] = testing.values * x_min_var
    monthly_returns[(12*i):(12*i + 12),2] = testing.values * x_factor3 
    monthly_returns[(12*i):(12*i + 12),3] = testing.values * x_factor5
    
    # Store the number of stocks selected for record keeping purposes
    num_min_var[i,0] = sum(x_min_var > 0)
    num_factor[i,0] = sum(x_factor3 > 0)
    
    print(str(start_year+5) + ' is complete')

Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1991 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1992 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1993 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1994 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1995 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1996 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1997 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1998 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
1999 is complete
Minimizing variance is complete
F-F 3 factors is complete
F-F 5 factors is complete
2000 is

In [15]:
col_names = ['Equal', 'Mean-variance', 'Three-Factor', 'Five-Factor']
returns = pd.DataFrame(monthly_returns, columns = col_names)

In [16]:
returns['Date'] = pd.date_range('1/1/1991', periods=12*25, freq='M')

In [17]:
returns['Period'] = 1
returns.loc[returns['Date'] >= pd.datetime(2011, 1, 1), 'Period'] = 5
returns.loc[(returns['Date'] >= pd.datetime(2006, 1, 1)) & (returns['Date'] < pd.datetime(2011, 1, 1)), 'Period'] = 4
returns.loc[(returns['Date'] >= pd.datetime(2001, 1, 1)) & (returns['Date'] < pd.datetime(2006, 1, 1)), 'Period'] = 3
returns.loc[(returns['Date'] >= pd.datetime(1996, 1, 1)) & (returns['Date'] < pd.datetime(2001, 1, 1)), 'Period'] = 2
returns.loc[(returns['Date'] >= pd.datetime(1991, 1, 1)) & (returns['Date'] < pd.datetime(1996, 1, 1)), 'Period'] = 1


In [35]:
returns.set_index('Date', inplace=True)
returns['year'] = returns.index.year
returns['month'] = returns.index.month
returns.head()

,Equal,Mean-variance,Three-Factor,Five-Factor,Period,year,month
Date,,,,,,,
1991-01-31,0.076230,-0.016445,0.053324,0.052417,1,1991,1
1991-02-28,0.101428,0.063873,0.080100,0.080025,1,1991,2
1991-03-31,0.043637,0.024132,0.033543,0.033102,1,1991,3
1991-04-30,0.018286,0.004339,0.010107,0.009918,1,1991,4
1991-05-31,0.040537,0.003743,0.036662,0.036018,1,1991,5


In [44]:
returns.pivot(index='month', columns='year', values='Equal')

year,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
month,,,,,,,,,,,,,,,,,,,,,
1,0.076230,0.062262,0.034495,0.038126,0.016477,0.012128,0.034313,-0.005418,-0.005042,-0.016583,...,0.066709,0.025779,-0.041117,-0.082845,-0.022294,0.015788,0.061654,0.063247,-0.028266,-0.025319
2,0.101428,0.029948,0.018877,-0.002983,0.045060,0.021389,0.003238,0.067639,-0.043922,0.003049,...,0.001371,0.000900,-0.020919,-0.125011,0.043589,0.037115,0.030140,0.013532,0.047091,0.051368
3,0.043637,-0.011005,0.036865,-0.041571,0.024691,0.018927,-0.021055,0.042724,0.002393,0.077114,...,0.034668,0.014626,0.003149,0.104935,0.071757,0.022068,0.025635,0.039297,0.015009,-0.001574
4,0.018286,0.001063,-0.016390,0.009211,0.021501,0.033674,0.014426,0.004357,0.092421,0.011736,...,0.016852,0.031757,0.039488,0.191689,0.055876,0.022345,-0.004358,0.000540,-0.009113,-0.005294
5,0.040537,0.021573,0.039465,-0.003970,0.031148,0.032209,0.077782,-0.032297,0.020519,0.000552,...,-0.035449,0.035708,0.034930,0.053219,-0.076309,-0.014338,-0.064756,0.032302,0.013929,0.006962
6,-0.030081,-0.021670,0.006672,-0.015133,0.031107,-0.010285,0.048984,-0.006146,0.044369,-0.000106,...,0.003473,-0.006082,-0.084365,0.016125,-0.056786,-0.012854,0.035932,-0.007704,0.034053,-0.012499
7,0.032004,0.036949,0.012355,0.020712,0.040164,-0.051725,0.070581,-0.059274,-0.004228,0.017579,...,-0.012263,-0.035599,0.016688,0.105169,0.070203,-0.031639,0.005450,0.061660,-0.044012,-0.018919
8,0.026838,-0.009478,0.044930,0.037552,0.018329,0.049951,0.010421,-0.133849,-0.027873,0.063176,...,0.021144,0.011456,0.029822,0.053555,-0.058791,-0.062347,0.025134,-0.028478,0.043706,-0.039893
9,-0.000863,0.022844,0.014921,-0.011467,0.030379,0.039903,0.062110,0.052144,-0.027274,-0.001420,...,0.013214,0.013322,-0.076977,0.059082,0.104574,-0.094012,0.030876,0.045363,-0.041131,-0.032402


In [46]:
# Write out monthly results
def write_monthly(df, metric):
    df = df.pivot(index='month', columns='year', values=metric)
    df.to_csv('josiah_' + metric + '.csv', index = False)
    
write_monthly(returns, 'Equal')
write_monthly(returns, 'Mean-variance')
write_monthly(returns, 'Three-Factor')
write_monthly(returns, 'Five-Factor')

In [18]:
def sharpe(x): 
    return(np.mean(x) / np.std(x))

In [19]:
def std(x):
    return(np.std(x, ddof = 1))

In [22]:
returns.groupby(['Period'])[col_names].agg([np.mean, std, sharpe])

Equal                     Mean-variance                      \
            mean       std    sharpe          mean       std    sharpe   
Period                                                                   
1       0.019642  0.027829  0.711758      0.011740  0.018734  0.631940   
2       0.015121  0.039315  0.387868      0.015265  0.031404  0.490205   
3       0.015015  0.041357  0.366130      0.016316  0.030980  0.531103   
4       0.010098  0.061710  0.165019      0.005420  0.036177  0.151096   
5       0.009719  0.038118  0.257126      0.013156  0.027029  0.490824   

       Three-Factor                     Five-Factor                      
               mean       std    sharpe        mean       std    sharpe  
Period                                                                   
1          0.015266  0.023908  0.643905    0.015196  0.023853  0.642465  
2          0.014348  0.037782  0.382956    0.014354  0.037917  0.381754  
3          0.012195  0.033204  0.370374    0.012174  0.033135  0.370505  
4          0.007766  0.053028  0.147685    0.007765  0.053046  0.147611  
5          0.010108  0.032716  0.311579    0.010153  0.032707  0.313054

In [23]:
returns.groupby([lambda idx: 'Total'])[col_names].agg([np.mean, np.std, sharpe])

Equal                     Mean-variance                      \
           mean       std    sharpe          mean       std    sharpe   
Total  0.013919  0.042977  0.324415      0.012379  0.029502  0.420317   

      Three-Factor                     Five-Factor                      
              mean       std    sharpe        mean       std    sharpe  
Total     0.011937  0.037225  0.321199    0.011928  0.037234  0.320898

In [24]:
returns.head()

,Equal,Mean-variance,Three-Factor,Five-Factor,Date,Period
0,0.076230,-0.016445,0.053324,0.052417,1991-01-31,1
1,0.101428,0.063873,0.080100,0.080025,1991-02-28,1
2,0.043637,0.024132,0.033543,0.033102,1991-03-31,1
3,0.018286,0.004339,0.010107,0.009918,1991-04-30,1
4,0.040537,0.003743,0.036662,0.036018,1991-05-31,1


In [ ]:
# Format and write the results
results = [format_results(results_train_equal, 0, 'Equal', stocks.shape[1])
          , format_results(results_train_min_var, 0, 'Mean-Variance', num_min_var)
           , format_results(results_train_factor, 0, 'Factor', num_factor)
          , format_results(results_test_equal, 1, 'Equal', stocks.shape[1])
          , format_results(results_test_min_var, 1, 'Mean-Variance', num_min_var)
          , format_results(results_test_factor, 1, 'Factor', num_factor)]
results = pd.concat(results)

In [ ]:
results.head()

In [ ]:
#results.to_csv('../../../data/factor_results.csv', index=False)